In [56]:
from flask import Flask
from flask import jsonify
from flask import request

app = Flask(__name__)

# Montagem inicial do grafo pode ser qualquer arquivo de input
dict_grafo = {
            'Carlos': ['Ana'],
            'Ana': ['Maria','Vinicius','Carlos','Joao'],
            'Maria': ['Ana','Vinicius'],
            'Vinicius': ['Ana','Maria'],
            'Joao': ['Ana','Luiza'],
            'Luiza': ['Joao'],
}

#novas pessoas para serem adicionadas ao grafo inicial
dict_novos_amigos = {
            'Fernando': ['Maria','Vinicius','Carlos','Joao'],
            'Gustavo' :['Fernando'],
}

#criação inicial dos nos do grafo e dos relacionamentos entre os nos
def monta_relacionamentos_nos(dicionario):

    relacionamentos = []
   
    for chave in dicionario.keys():
                
        for vertice in dicionario[chave]:
            relacionamentos.append((chave, vertice))

    return relacionamentos

relacionamentos = monta_relacionamentos_nos(dict_grafo)
class Grafo(object):
    @app.route('/',methods=['GET'])           
    def todos():
        nos_grafo = []
        for indice in dict_grafo.keys():
            nos_grafo.append(indice)
        return jsonify(nos_grafo), 200


    @app.route('/relacionamentos/<string:pessoa>',methods=['GET'])           
    def conhecidos(pessoa):

        nos_grafo = []
        for indice in relacionamentos: 
            if indice[0] == pessoa:
                nos_grafo.append (indice[1])

        if nos_grafo != []:    
            return jsonify(nos_grafo), 200
        else:
            return jsonify("Error: Not Found"), 404

    @app.route('/naoamigos/<string:pessoa>',methods=['GET'])    
    def amigos_dos_amigos (pessoa):

        nao_amigos = list()
        list_amigos = []

        nos_grafo = []
        for indice in dict_grafo.keys():
            nos_grafo.append(indice)

        for indice in relacionamentos: 
            if indice[0] == pessoa:
                list_amigos.append (indice[1])

        if list_amigos == []:    
            return jsonify("Error: Not Found"), 404
        else:
            for chave in nos_grafo:
                if chave != pessoa and chave not in list_amigos:
                    nao_amigos.append(chave)

            if nao_amigos != []:    
                return jsonify(nao_amigos), 200
            else:
                return jsonify("Todos são amigos"), 200

    @app.route('/novo',methods=['PUT'])           
    def adiciona():

        nos_grafo = []
        for indice in dict_grafo.keys():
            nos_grafo.append(indice)

        for indice in dict_novos_amigos.keys():
            if indice in nos_grafo:
                return jsonify ("Pessoa já faz parte dos relacionamentos"),202
            else:
                for chave in dict_novos_amigos[indice]:
                    if chave not in nos_grafo:
                        return jsonify("Amigo não faz parte do relacionamento"), 404

                    relacionamentos.append((indice, chave))

            nos_grafo.append(indice)
            dict_grafo.update(dict_novos_amigos)
        return jsonify(dict_grafo), 200        


    if __name__ == '__main__':
     app.run()
    
grafo=Grafo(relacionamentos)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2022 20:10:37] "GET / HTTP/1.1" 200 -


TypeError: Grafo() takes no arguments